In [63]:
from PIL import Image
directory = 'test4/'
fileName = 'test'
extension = '.gif'
sizex = 50
sizey = 25

file = open(directory+fileName+"-pixel-map.txt", "a")
file.truncate(0) # need '0' when using r+

im = Image.open(directory+fileName+extension)

def iter_frames(im):
    try:
        i= 0
        while 1:
            im.seek(i)
            imframe = im.copy()
            if i == 0: 
                palette = imframe.getpalette()
            else:
                imframe.putpalette(palette)
            yield imframe
            i += 1
    except EOFError:
        pass

file.write('local video = {\n')
for i, frame in enumerate(iter_frames(im)):
    filename = (directory+'frame%d'+'.png') % i
    frame = frame.convert(mode='P', palette=Image.ADAPTIVE)
    frame = frame.resize((sizex,sizey), Image.ANTIALIAS)
    frame = frame.convert('RGB')
    quality_val = 90
    frame.save(filename, 'PNG', quality=quality_val)
    capture = Image.open(filename)
    file.write('\tframe'+str(i+1)+' = {\n')
    for x in range(0, sizex):
        file.write('\t{')
        for y in range(0, sizey):
            if y == sizey-1:
                file.write(str(capture.getpixel((x,y))).strip('()'))
            else:
                file.write(str(capture.getpixel((x,y))).strip('()')+', ')
        if x == sizex-1:
            file.write('}\n')
        else:
            file.write('},\n')
    if i+1 == len(list(iter_frames(im))):
        file.write('\t}\n')
    else:
        file.write('\t},\n\n')

file.write('} return video')
       
file.close()
print('Finished making output table: '+fileName+"-pixel-map.txt")

Finished making output table: test-pixel-map.txt


In [83]:
import cv2
from PIL import Image
import time
directory = 'rainingTacos/'
vidcap = cv2.VideoCapture(directory+'rainingTacos.mp4')
success,image = vidcap.read()
count = 0

start_time = time.time()

sizex = 100
sizey = 50


file = open(directory+fileName+"-pixel-map.txt", "a")
file.truncate(0) # need '0' when using r+
file.write('local video = {}\n')

print('Processing video...')

while success:
    count += 1
    filename = directory+"frame%d.png" % count
    cv2.imwrite(filename, image)     # save frame as JPEG file      
    success,image = vidcap.read()
    capture = Image.open(filename)  
    capture = capture.resize((sizex,sizey), Image.ANTIALIAS)
    capture = capture.convert('RGB')
    capture.save(filename, 'PNG')
    file.write('\tvideo["frame'+str(count)+'"] = {\n')
    for x in range(0, sizex):
        file.write('\t{')
        for y in range(0, sizey):
            if y == sizey-1:
                file.write(str(capture.getpixel((x,y))).strip('()'))
            else:
                file.write(str(capture.getpixel((x,y))).strip('()')+', ')
        if x == sizex-1:
            file.write('}\n')
        else:
            file.write('},\n')
    file.write('\t}\n')
    capture.close()    

file.write('return video')
file.close()

print('Video finished processing into rgb framed table.')
print('Number of frames: ', count)
print("--- %s seconds ---" % (time.time() - start_time))

Processing video...
Video finished processing into rgb framed table.
Number of frames:  1866
--- 46.801599979400635 seconds ---
